In [35]:
!pip install gensim --user
!pip install textblob

In [113]:
import pandas as pd
import numpy as np
import re

#Timing
from tqdm import tqdm

#Word Embeddings
import nltk
from textblob import TextBlob
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#Models
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Cameron
[nltk_data]     Milne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Cameron
[nltk_data]     Milne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## EDA

In [114]:
scored_tweets = pd.read_csv('scored_tweets.csv')

In [115]:
scored_tweets.columns

Index(['created_at', 'text', 'username', 'screen_name', 'verified',
       'followers_count', 'TOXICITY', 'INSULT', 'PROFANITY', 'THREAT',
       'SEXUALLY_EXPLICIT'],
      dtype='object')

In [116]:
scored_tweets.corr()

,TOXICITY,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT
TOXICITY,1.000000,0.948310,0.923765,0.555334,0.674901
INSULT,0.948310,1.000000,0.837267,0.446485,0.531912
PROFANITY,0.923765,0.837267,1.000000,0.435503,0.733206
THREAT,0.555334,0.446485,0.435503,1.000000,0.434425
SEXUALLY_EXPLICIT,0.674901,0.531912,0.733206,0.434425,1.000000


In [117]:
scored_tweets.head(3)

,created_at,text,username,screen_name,verified,followers_count,TOXICITY,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT
0,created_at,text,username,screen_name,verified,followers_count,0.042519,0.015740,0.037332,0.043029,0.040426
1,Thu Nov 11 18:25:52 +0000 2021,@fetuslhlvr we aren't familiar with each other...,rayn,faultyplacebo,False,172,0.112422,0.082374,0.096755,0.115139,0.106634
2,Thu Nov 11 18:25:52 +0000 2021,@BallBlastEm I wouldn't trust him as my primar...,Bryan,BryanOreally,False,1467,0.082990,0.073394,0.067158,0.077337,0.075090


## New Preprocessing

In [118]:
scored_tweets.dropna(subset=['text'], inplace=True)

In [119]:
from nltk.tokenize import sent_tokenize, word_tokenize

#tokenizes every tweet
tokenizer = nltk.RegexpTokenizer(r"\w+")
scored_tweets['processed'] = scored_tweets['text'].apply(tokenizer.tokenize)
scored_tweets['processed']

0                                                  [text]
1       [fetuslhlvr, we, aren, t, familiar, with, each...
2       [BallBlastEm, I, wouldn, t, trust, him, as, my...
3       [I, am, not, feeling, exhausted, today, for, o...
4       [On, this, Remembrance, Day, I, ve, been, to, ...
                              ...                        
9793    [MorphingReality, JeremyDBoreing, realDailyWir...
9794    [Just, got, a, call, from, my, maid, Afreen, w...
9795    [Hey, guysMy, friend, is, looking, for, a, cle...
9796    [So, good, I, had, to, share, Check, out, all,...
9797    [kogantwerkteam, also, the, only, news, is, Ca...
Name: processed, Length: 9798, dtype: object

In [120]:
from nltk.corpus import stopwords 

#Widely used stopword library
stop_words = set(stopwords.words('english'))

#gets rid of numbers
stop_words.update([str(x) for x in np.arange(10)])

#apply to tokens
scored_tweets['processed'] = scored_tweets.processed.apply(lambda x: [i for i in x if not i in stop_words])
scored_tweets['processed']

0                                                  [text]
1       [fetuslhlvr, familiar, need, someone, confide,...
2       [BallBlastEm, I, trust, primary, backup, I, th...
3                          [I, feeling, exhausted, today]
4       [On, Remembrance, Day, I, service, WellsCathed...
                              ...                        
9793    [MorphingReality, JeremyDBoreing, realDailyWir...
9794    [Just, got, call, maid, Afreen, bursted, crack...
9795    [Hey, guysMy, friend, looking, clean, cool, fe...
9796    [So, good, I, share, Check, items, I, loving, ...
9797    [kogantwerkteam, also, news, Carlos, IG, Q, am...
Name: processed, Length: 9798, dtype: object

In [121]:
scored_tweets['processed'] = [' '.join(l) for l in scored_tweets['processed']]
scored_tweets['processed']

0                                                    text
1       fetuslhlvr familiar need someone confide curre...
2       BallBlastEm I trust primary backup I think may...
3                               I feeling exhausted today
4       On Remembrance Day I service WellsCathedral1 B...
                              ...                        
9793    MorphingReality JeremyDBoreing realDailyWire Y...
9794    Just got call maid Afreen bursted crackers Kar...
9795    Hey guysMy friend looking clean cool female sh...
9796    So good I share Check items I loving Poshmarka...
9797    kogantwerkteam also news Carlos IG Q amp A say...
Name: processed, Length: 9798, dtype: object

## Models

In [132]:
!pip install lightgbm
!pip install hyperopt

In [122]:
def split_data(df=scored_tweets, feature='processed', label='TOXICITY'):
    from sklearn.model_selection import train_test_split
    
    train_data, test_data = train_test_split(df, test_size=0.25, random_state=7)
    X_train, X_test = np.asarray(train_data['processed']), np.asarray(test_data['processed'])
    y_train, y_test = np.asarray(train_data[label]), np.asarray(test_data[label])
    
    return X_train, X_test, y_train, y_test

In [123]:
def TFIDF(X_train, X_test):
    from sklearn.feature_extraction.text import TfidfVectorizer

    vectorizer = TfidfVectorizer()
    train_tfidf = vectorizer.fit_transform(X_train)
    test_tfidf = vectorizer.transform(X_test)
    
    return train_tfidf, test_tfidf    

In [137]:
def LinearRegression(feature_category):
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import r2_score
    from sklearn.metrics import accuracy_score
    
    X_train, X_test, y_train, y_test = split_data(label=feature_category)
    train_tfidf, test_tfidf = TFIDF(X_train, X_test)

    model = LinearRegression().fit(train_tfidf, y_train)
    y_pred = model.predict(test_tfidf)
    
    mae = round(mean_absolute_error(y_pred, y_test), 4)
    r2 = round(r2_score(y_pred, y_test), 4)
    
    return (r2, mae, model)

In [136]:
def LightGBM(feature_category):
    import lightgbm as lgb
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import r2_score
    
    X_train, X_test, y_train, y_test = split_data(label=feature_category)
    train_tfidf, test_tfidf = TFIDF(X_train, X_test)
    
    hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l1','l2'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000
    }
    
    gbm = lgb.LGBMRegressor(**hyper_params)
    gbm.fit(train_tfidf, y_train, eval_set=[(test_tfidf, y_test)], eval_metric='l1', early_stopping_rounds=1000)
    y_pred = gbm.predict(test_tfidf, num_iteration=gbm.best_iteration_)
    
    mae = round(mean_absolute_error(y_pred, y_test), 4)
    r2 = round(r2_score(y_pred, y_test), 4)
    
    return (r2, mae, gbm)

In [135]:
def SVR(feature_category):
    from sklearn.svm import SVR
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import r2_score
    
    X_train, X_test, y_train, y_test = split_data(label=feature_category)
    train_tfidf, test_tfidf = TFIDF(X_train, X_test)
    
    svr = SVR(kernel='linear').fit(train_tfidf, y_train)
    y_pred = svr.predict(test_tfidf)
    
    mae = round(mean_absolute_error(y_pred, y_test), 4)
    r2 = round(r2_score(y_pred, y_test), 4) 
    
    return (r2, mae, svr)

In [153]:
def ElasticNet(feature_category):
    from sklearn.linear_model import ElasticNet
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import r2_score
    
    X_train, X_test, y_train, y_test = split_data(label=feature_category)
    train_tfidf, test_tfidf = TFIDF(X_train, X_test)
    
    model = ElasticNet(random_state=0).fit(train_tfidf, y_train)
    y_pred = model.predict(test_tfidf)
    
    mae = round(mean_absolute_error(y_pred, y_test), 4)
    r2 = round(r2_score(y_pred, y_test), 4) 
    
    return (r2, mae, model)

### Toxicity

In [154]:
category = 'TOXICITY'

lr_results = LinearRegression(category)
gbm_results = LightGBM(category)
svr_results = SVR(category)
en_results = ElasticNet(category)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029916 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[59]	valid_0's l1: 0.157826	valid_0's l2: 0.0425063
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[60]	valid_0's l1: 0.157662	valid_0's l2: 0.0423911
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[61]	valid_0's l1: 0.157513	valid_0's l2: 0.0422971
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[62]	valid_0's l1: 0.157367	valid_0's l2: 0.0422032
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[63]	valid_0's l1: 0.157212	valid_0's l2: 0.0421097
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[64]	valid_0's l1: 0.157085	valid_0's l2: 0.042033
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[65]	valid_0's l1: 0.156938	valid_0's l2: 0.0419403
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[66]	valid_0's l1: 0.156794	valid_0's

[138]	valid_0's l1: 0.148047	valid_0's l2: 0.0368696
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[139]	valid_0's l1: 0.147936	valid_0's l2: 0.0368116
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[140]	valid_0's l1: 0.147825	valid_0's l2: 0.0367558
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[141]	valid_0's l1: 0.147736	valid_0's l2: 0.0367149
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[142]	valid_0's l1: 0.147647	valid_0's l2: 0.0366732
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[143]	valid_0's l1: 0.147574	valid_0's l2: 0.036642
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[144]	valid_0's l1: 0.147469	valid_0's l2: 0.036592
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[145]	valid_0's l1: 0.147367	valid_0's l2: 0.0365432
[LightGBM] [Warning] No further splits with positive 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[226]	valid_0's l1: 0.140371	valid_0's l2: 0.0334754
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[227]	valid_0's l1: 0.140301	valid_0's l2: 0.0334503
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[228]	valid_0's l1: 0.140243	valid_0's l2: 0.0334266
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[229]	valid_0's l1: 0.140188	valid_0's l2: 0.033407
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[230]	valid_0's l1: 0.140119	valid_0's l2: 0.0333826
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[231]	valid_0's l1: 0.140048	valid_0's l2: 0.0333532
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[232]	valid_0's l1: 0.139995	valid_0's l2: 0.0333276
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[233]	valid_0's l1: 0.13992	va

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3050]	valid_0's l1: 0.11957	valid_0's l2: 0.0275924
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3051]	valid_0's l1: 0.119569	valid_0's l2: 0.0275915
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3052]	valid_0's l1: 0.11957	valid_0's l2: 0.027591
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3053]	valid_0's l1: 0.119571	valid_0's l2: 0.0275901
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3054]	valid_0's l1: 0.119573	valid_0's l2: 0.0275897
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3055]	valid_0's l1: 0.119572	valid_0's l2: 0.0275887
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3056]	valid_0's l1: 0.119571	valid_0's l2: 0.0275879
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3057]	valid_0's l1: 0.11

[3141]	valid_0's l1: 0.119529	valid_0's l2: 0.0276003
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3142]	valid_0's l1: 0.119528	valid_0's l2: 0.0275996
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3143]	valid_0's l1: 0.119526	valid_0's l2: 0.0275989
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3144]	valid_0's l1: 0.119524	valid_0's l2: 0.0275982
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3145]	valid_0's l1: 0.119522	valid_0's l2: 0.0275976
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3146]	valid_0's l1: 0.11952	valid_0's l2: 0.0275968
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3147]	valid_0's l1: 0.119518	valid_0's l2: 0.0275959
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3148]	valid_0's l1: 0.119517	valid_0's l2: 0.0275952
[LightGBM] [Warning] No further splits with 

[3231]	valid_0's l1: 0.119482	valid_0's l2: 0.0276045
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3232]	valid_0's l1: 0.119483	valid_0's l2: 0.0276044
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3233]	valid_0's l1: 0.119482	valid_0's l2: 0.0276042
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3234]	valid_0's l1: 0.119481	valid_0's l2: 0.0276036
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3235]	valid_0's l1: 0.119483	valid_0's l2: 0.0276045
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3236]	valid_0's l1: 0.119481	valid_0's l2: 0.0276041
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3237]	valid_0's l1: 0.119483	valid_0's l2: 0.0276049
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3238]	valid_0's l1: 0.119479	valid_0's l2: 0.0276045
[LightGBM] [Warning] No further splits with

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3321]	valid_0's l1: 0.119413	valid_0's l2: 0.0276006
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3322]	valid_0's l1: 0.119415	valid_0's l2: 0.0276
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3323]	valid_0's l1: 0.119418	valid_0's l2: 0.0275997
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3324]	valid_0's l1: 0.119421	valid_0's l2: 0.0275992
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3325]	valid_0's l1: 0.119424	valid_0's l2: 0.0275998
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3326]	valid_0's l1: 0.119426	valid_0's l2: 0.0275995
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3327]	valid_0's l1: 0.11943	valid_0's l2: 0.0276001
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3328]	valid_0's l1: 0.119

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3413]	valid_0's l1: 0.119455	valid_0's l2: 0.0276007
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3414]	valid_0's l1: 0.119454	valid_0's l2: 0.0276003
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3415]	valid_0's l1: 0.119452	valid_0's l2: 0.0275999
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3416]	valid_0's l1: 0.119453	valid_0's l2: 0.0276003
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3417]	valid_0's l1: 0.119452	valid_0's l2: 0.0275999
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3418]	valid_0's l1: 0.119453	valid_0's l2: 0.0275997
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3419]	valid_0's l1: 0.119454	valid_0's l2: 0.0275992
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3420]	valid_0's l1: 0

In [155]:
print("Toxicity Results")
print("================")
print("[r2_score, mean_absolute_error, model]")
print("================")
for model in [lr_results, gbm_results, svr_results, en_results]:
    print(model)

Toxicity Results
[r2_score, mean_absolute_error, model]
(0.3546, 0.1179, LinearRegression())
(-0.3494, 0.12, LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, feature_fraction=0.9,
              learning_rate=0.005, max_bin=512, max_depth=8,
              metric=['l1', 'l2'], num_iterations=100000, num_leaves=128,
              objective='regression', task='train', verbose=0))
(0.1261, 0.1205, SVR(kernel='linear'))
(-6.430387922879726e+31, 0.1671, ElasticNet(random_state=0))


### Profanity

In [150]:
category = 'PROFANITY'

lr_results = LinearRegression(category)
gbm_results = LightGBM(category)
svr_results = SVR(category)
en_results = ElasticNet(category)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035923 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[89]	valid_0's l1: 0.134331	valid_0's l2: 0.0367312
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[90]	valid_0's l1: 0.134176	valid_0's l2: 0.0366235
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[91]	valid_0's l1: 0.134029	valid_0's l2: 0.0365234
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[92]	valid_0's l1: 0.133852	valid_0's l2: 0.0364036
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[93]	valid_0's l1: 0.133709	valid_0's l2: 0.0363122
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[94]	valid_0's l1: 0.133577	valid_0's l2: 0.0362191
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[95]	valid_0's l1: 0.133402	valid_0's l2: 0.0361023
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[96]	valid_0's l1: 0.133242	valid_0'

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[163]	valid_0's l1: 0.12533	valid_0's l2: 0.0313995
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[164]	valid_0's l1: 0.125223	valid_0's l2: 0.0313401
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[165]	valid_0's l1: 0.125133	valid_0's l2: 0.0312943
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[166]	valid_0's l1: 0.125027	valid_0's l2: 0.0312359
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[167]	valid_0's l1: 0.124939	valid_0's l2: 0.0311833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[168]	valid_0's l1: 0.12485	valid_0's l2: 0.0311389
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[169]	valid_0's l1: 0.12474	valid_0's l2: 0.03108
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[170]	valid_0's l1: 0.124636	valid

[235]	valid_0's l1: 0.118885	valid_0's l2: 0.0284241
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[236]	valid_0's l1: 0.118814	valid_0's l2: 0.0283939
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[237]	valid_0's l1: 0.11874	valid_0's l2: 0.0283633
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[238]	valid_0's l1: 0.118667	valid_0's l2: 0.0283336
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[239]	valid_0's l1: 0.118591	valid_0's l2: 0.0283025
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[240]	valid_0's l1: 0.118515	valid_0's l2: 0.0282718
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[241]	valid_0's l1: 0.118446	valid_0's l2: 0.0282468
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[242]	valid_0's l1: 0.118377	valid_0's l2: 0.0282221
[LightGBM] [Warning] No further splits with positive

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2601]	valid_0's l1: 0.102842	valid_0's l2: 0.0235071
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2602]	valid_0's l1: 0.102839	valid_0's l2: 0.0235059
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2603]	valid_0's l1: 0.102836	valid_0's l2: 0.0235048
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2604]	valid_0's l1: 0.102834	valid_0's l2: 0.0235037
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2605]	valid_0's l1: 0.102831	valid_0's l2: 0.0235027
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2606]	valid_0's l1: 0.102829	valid_0's l2: 0.0235019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2607]	valid_0's l1: 0.102826	valid_0's l2: 0.0235013
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2608]	valid_0's l1: 0

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2684]	valid_0's l1: 0.102809	valid_0's l2: 0.0234883
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2685]	valid_0's l1: 0.102809	valid_0's l2: 0.0234873
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2686]	valid_0's l1: 0.102807	valid_0's l2: 0.0234863
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2687]	valid_0's l1: 0.102805	valid_0's l2: 0.0234859
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2688]	valid_0's l1: 0.102803	valid_0's l2: 0.023485
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2689]	valid_0's l1: 0.102802	valid_0's l2: 0.0234847
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2690]	valid_0's l1: 0.102802	valid_0's l2: 0.0234851
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2691]	valid_0's l1: 0.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2753]	valid_0's l1: 0.102807	valid_0's l2: 0.0235031
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2754]	valid_0's l1: 0.102806	valid_0's l2: 0.0235043
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2755]	valid_0's l1: 0.102806	valid_0's l2: 0.0235057
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2756]	valid_0's l1: 0.102805	valid_0's l2: 0.023506
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2757]	valid_0's l1: 0.102805	valid_0's l2: 0.0235074
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2758]	valid_0's l1: 0.102805	valid_0's l2: 0.0235085
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2759]	valid_0's l1: 0.102806	valid_0's l2: 0.0235099
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2760]	valid_0's l1: 0.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2826]	valid_0's l1: 0.102811	valid_0's l2: 0.0235282
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2827]	valid_0's l1: 0.102815	valid_0's l2: 0.0235284
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2828]	valid_0's l1: 0.102817	valid_0's l2: 0.0235284
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2829]	valid_0's l1: 0.102821	valid_0's l2: 0.0235293
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2830]	valid_0's l1: 0.102823	valid_0's l2: 0.0235293
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2831]	valid_0's l1: 0.102826	valid_0's l2: 0.0235302
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2832]	valid_0's l1: 0.102826	valid_0's l2: 0.0235302
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2833]	valid_0's l1: 0

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2910]	valid_0's l1: 0.102765	valid_0's l2: 0.0235124
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2911]	valid_0's l1: 0.102766	valid_0's l2: 0.0235119
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2912]	valid_0's l1: 0.102767	valid_0's l2: 0.0235116
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2913]	valid_0's l1: 0.10277	valid_0's l2: 0.0235116
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2914]	valid_0's l1: 0.10277	valid_0's l2: 0.0235105
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2915]	valid_0's l1: 0.102773	valid_0's l2: 0.0235108
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2916]	valid_0's l1: 0.102777	valid_0's l2: 0.0235108
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2917]	valid_0's l1: 0.1

In [151]:
print("Profanity Results")
print("================")
print("[r2_score, mean_absolute_error, model]")
print("================")
for model in [lr_results, gbm_results, svr_results, en_results]:
    print(model)

Profanity Results
[r2_score, mean_absolute_error, model]
(0.3789, 0.1133, LinearRegression())
(-0.0141, 0.1027, LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, feature_fraction=0.9,
              learning_rate=0.005, max_bin=512, max_depth=8,
              metric=['l1', 'l2'], num_iterations=100000, num_leaves=128,
              objective='regression', task='train', verbose=0))
(0.229, 0.1144, SVR(kernel='linear'))
(-6.336440349505237e+31, 0.1494, ElasticNet(random_state=0))


### Sexually Explicit

In [148]:
category = 'SEXUALLY_EXPLICIT'

lr_results = LinearRegression(category)
gbm_results = LightGBM(category)
svr_results = SVR(category)
en_results = ElasticNet(category)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025817 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[72]	valid_0's l1: 0.110306	valid_0's l2: 0.0297162
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[73]	valid_0's l1: 0.110231	valid_0's l2: 0.0296656
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[74]	valid_0's l1: 0.110155	valid_0's l2: 0.0296145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[75]	valid_0's l1: 0.110082	valid_0's l2: 0.0295649
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[76]	valid_0's l1: 0.110009	valid_0's l2: 0.0295158
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[77]	valid_0's l1: 0.109936	valid_0's l2: 0.0294672
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[78]	valid_0's l1: 0.109864	valid_0's l2: 0.0294192
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[79]	valid_0's l1: 0.109792	valid_0'

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[136]	valid_0's l1: 0.106486	valid_0's l2: 0.027345
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[137]	valid_0's l1: 0.106435	valid_0's l2: 0.0273158
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[138]	valid_0's l1: 0.106393	valid_0's l2: 0.0272911
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[139]	valid_0's l1: 0.106343	valid_0's l2: 0.0272624
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[140]	valid_0's l1: 0.106291	valid_0's l2: 0.0272338
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[141]	valid_0's l1: 0.106257	valid_0's l2: 0.0272139
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[142]	valid_0's l1: 0.106213	valid_0's l2: 0.0271869
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[143]	valid_0's l1: 0.106182	v

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[206]	valid_0's l1: 0.103536	valid_0's l2: 0.0258135
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[207]	valid_0's l1: 0.103492	valid_0's l2: 0.0257964
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[208]	valid_0's l1: 0.103451	valid_0's l2: 0.0257804
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[209]	valid_0's l1: 0.103418	valid_0's l2: 0.0257664
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[210]	valid_0's l1: 0.103378	valid_0's l2: 0.0257491
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[211]	valid_0's l1: 0.103345	valid_0's l2: 0.0257342
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[212]	valid_0's l1: 0.103311	valid_0's l2: 0.0257187
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[213]	valid_0's l1: 0.103264	

...

[3818]	valid_0's l1: 0.0950469	valid_0's l2: 0.0221147
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3819]	valid_0's l1: 0.0950445	valid_0's l2: 0.0221139
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3820]	valid_0's l1: 0.0950466	valid_0's l2: 0.0221147
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3821]	valid_0's l1: 0.0950483	valid_0's l2: 0.0221146
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3822]	valid_0's l1: 0.0950512	valid_0's l2: 0.0221146
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3823]	valid_0's l1: 0.0950531	valid_0's l2: 0.0221145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3824]	valid_0's l1: 0.0950548	valid_0's l2: 0.0221147
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3825]	valid_0's l1: 0.0950571	valid_0's l2: 0.0221145
[LightGBM] [Warning] No further spl

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3882]	valid_0's l1: 0.0951086	valid_0's l2: 0.0221335
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3883]	valid_0's l1: 0.09511	valid_0's l2: 0.0221335
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3884]	valid_0's l1: 0.0951154	valid_0's l2: 0.0221342
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3885]	valid_0's l1: 0.0951167	valid_0's l2: 0.0221342
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3886]	valid_0's l1: 0.0951214	valid_0's l2: 0.0221346
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3887]	valid_0's l1: 0.0951253	valid_0's l2: 0.0221345
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3888]	valid_0's l1: 0.0951266	valid_0's l2: 0.0221345
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3889]	valid_0's 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3964]	valid_0's l1: 0.095135	valid_0's l2: 0.0221286
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3965]	valid_0's l1: 0.0951384	valid_0's l2: 0.0221289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3966]	valid_0's l1: 0.0951419	valid_0's l2: 0.0221293
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3967]	valid_0's l1: 0.0951453	valid_0's l2: 0.0221296
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3968]	valid_0's l1: 0.0951507	valid_0's l2: 0.0221307
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3969]	valid_0's l1: 0.0951574	valid_0's l2: 0.0221322
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3970]	valid_0's l1: 0.0951612	valid_0's l2: 0.0221326
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3971]	valid_0's

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4051]	valid_0's l1: 0.0951141	valid_0's l2: 0.0221408
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4052]	valid_0's l1: 0.0951045	valid_0's l2: 0.0221404
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4053]	valid_0's l1: 0.0950955	valid_0's l2: 0.02214
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4054]	valid_0's l1: 0.0950875	valid_0's l2: 0.0221396
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4055]	valid_0's l1: 0.0950794	valid_0's l2: 0.0221396
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4056]	valid_0's l1: 0.095071	valid_0's l2: 0.0221394
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4057]	valid_0's l1: 0.0950624	valid_0's l2: 0.0221391
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4058]	valid_0's l

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4137]	valid_0's l1: 0.095114	valid_0's l2: 0.0221242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4138]	valid_0's l1: 0.0951158	valid_0's l2: 0.0221249
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4139]	valid_0's l1: 0.095119	valid_0's l2: 0.0221256
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4140]	valid_0's l1: 0.0951226	valid_0's l2: 0.0221262
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4141]	valid_0's l1: 0.0951238	valid_0's l2: 0.0221257
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4142]	valid_0's l1: 0.0951261	valid_0's l2: 0.0221255
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4143]	valid_0's l1: 0.0951292	valid_0's l2: 0.0221249
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4144]	valid_0's 

In [149]:
print("Sexually Explicit Results")
print("================")
print("[r2_score, mean_absolute_error, model]")
print("================")
for model in [lr_results, gbm_results, svr_results, en_results]:
    print(model)

Sexually Explicit Results
[r2_score, mean_absolute_error, model]
(0.2918, 0.0951, LinearRegression())
(-0.7877, 0.0952, LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, feature_fraction=0.9,
              learning_rate=0.005, max_bin=512, max_depth=8,
              metric=['l1', 'l2'], num_iterations=100000, num_leaves=128,
              objective='regression', task='train', verbose=0))
(0.009, 0.1026, SVR(kernel='linear'))
(0.0, 0.1167, ElasticNet(random_state=0))
